In [39]:
import sqlalchemy as sa

In [40]:
engine = sa.create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)

In [41]:
mapper_registry = sa.orm.registry()
Base = mapper_registry.generate_base()


class User(Base):
    __tablename__ = "user_account"
    id = sa.Column(sa.Integer, primary_key=True)
    name = sa.Column(sa.String(30))
    fullname = sa.Column(sa.String)
    addresses = sa.orm.relationship("Address", back_populates="user")

    def __repr__(self):
        return (
            f"User(id={self.id},"
            f" name={self.name},"
            f" fullname={self.fullname})"
        )


class Address(Base):
    __tablename__ = "address"
    id = sa.Column(sa.Integer, primary_key=True)
    email_address = sa.Column(sa.String, nullable=False)
    user_id = sa.Column(sa.Integer, sa.ForeignKey("user_account.id"))
    user = sa.orm.relationship("User", back_populates="addresses")

    def __repr__(self):
        return f"Address(id={self.id}," f" email_address=" f"{self.email_address})"

In [42]:
mapper_registry.metadata.create_all(engine)

2022-04-03 21:05:53,268 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 21:05:53,269 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-04-03 21:05:53,272 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 21:05:53,278 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-04-03 21:05:53,279 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 21:05:53,285 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-04-03 21:05:53,288 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 21:05:53,292 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-04-03 21:05:53,293 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 21:05:53,298 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-04-03 21:05:53,301 INFO sqlalchemy.engine.Engine [no key 0.00283s] ()
2022-04-03 21:05:53,304 INFO sqlalchemy.engine.Engine 
C

In [43]:
stmt = sa.insert(User).\
    values(name='spongebob', 
    fullname="Spongebob Squarepants")

In [44]:
print(stmt)

INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)


In [45]:
compiled = stmt.compile()

In [46]:
compiled.params

{'name': 'spongebob', 'fullname': 'Spongebob Squarepants'}

In [47]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

2022-04-03 21:05:59,708 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 21:05:59,710 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-04-03 21:05:59,713 INFO sqlalchemy.engine.Engine [generated in 0.00467s] ('spongebob', 'Spongebob Squarepants')
2022-04-03 21:05:59,718 INFO sqlalchemy.engine.Engine COMMIT


In [48]:
with engine.connect() as conn:
    result = conn.execute(
        sa.insert(User),
        [
            {"name": "sandy", 
            "fullname": "Sandy Cheeks"},
            {"name": "patrick", 
            "fullname": "Patrick Star"}
        ]
    )
    conn.commit()

2022-04-03 21:06:01,090 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 21:06:01,092 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-04-03 21:06:01,093 INFO sqlalchemy.engine.Engine [generated in 0.00236s] (('sandy', 'Sandy Cheeks'), ('patrick', 'Patrick Star'))
2022-04-03 21:06:01,098 INFO sqlalchemy.engine.Engine COMMIT


In [49]:
User.id

In [35]:
User.name

In [50]:
scalar_subq = (
    sa.select(User.id)
    .where(User.name == sa.bindparam("username"))
    .scalar_subquery()
)

with engine.connect() as conn:
    result = conn.execute(
        sa.insert(Address).values(user_id=scalar_subq),
        [
            {"username": "spongebob", "email_address": "spongebob@sqlalchemy.org"},
            {"username": "sandy", "email_address": "sandy@sqlalchemy.org"},
            {"username": "sandy", "email_address": "sandy@squirrelpower.org"},
        ],
    )
    conn.commit()

2022-04-03 21:06:08,382 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 21:06:08,385 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, (SELECT user_account.id 
FROM user_account 
WHERE user_account.name = ?))
2022-04-03 21:06:08,388 INFO sqlalchemy.engine.Engine [generated in 0.00642s] (('spongebob@sqlalchemy.org', 'spongebob'), ('sandy@sqlalchemy.org', 'sandy'), ('sandy@squirrelpower.org', 'sandy'))
2022-04-03 21:06:08,392 INFO sqlalchemy.engine.Engine COMMIT


In [51]:
stmt = sa.select(User).where(User.name == 'spongebob')
with sa.orm.Session(engine) as session:
    for row in session.execute(stmt):
        print(row)

2022-04-03 21:06:12,538 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 21:06:12,542 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-04-03 21:06:12,547 INFO sqlalchemy.engine.Engine [generated in 0.00514s] ('spongebob',)
(User(id=1, name=spongebob, fullname=Spongebob Squarepants),)
2022-04-03 21:06:12,551 INFO sqlalchemy.engine.Engine ROLLBACK
